In [1]:
import sys, os
project_dir = os.path.abspath('..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

import datetime
from src.predictionModule.FilterSamples import FilterSamples
from src.predictionModule.LoadupSamples import LoadupSamples

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]
# Usage
logger.info("This will print to the notebook's output cell")


This will print to the notebook's output cell


In [2]:
params = {
    "daysAfterPrediction": None,
    "idxAfterPrediction": 5,
    'timesteps': 60,
    'target_option': 'mean',
    
    "LoadupSamples_tree_scaling_standard": True,
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 10,

    "FilterSamples_q_up": 0.50,

    "FilterSamples_lincomb_epochs": 5,
    "FilterSamples_lincomb_show_progress": False,
    "FilterSamples_lincomb_featureratio": 0.5,
    "FilterSamples_lincomb_itermax": 2,
    "FilterSamples_lincomb_init_toprand":  1,

    "FilterSamples_days_to_train_end": 10,
    "FilterSamples_cat_over20": True,
    "FilterSamples_cat_posOneYearReturn": False,
    "FilterSamples_cat_posFiveYearReturn": False,
    "FilterSamples_taylor_horizon_days": 20,
    "FilterSamples_taylor_roll_window_days": 20
}

In [3]:
timegroup = "group_regOHLCV_over5years"
treegroup = "group_debug"

eval_date = datetime.date(year=2025, month=6, day=13)
evaldates = [eval_date - datetime.timedelta(days=i) for i in range(1, 6)]
start_train_date = datetime.date(year=2023, month=1, day=1)
split_Date = datetime.date(year=2025, month=1, day=1)
ls = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=treegroup,
    timegroup=timegroup,
    params=params,
)
ls.load_samples(main_path = "../src/featureAlchemy/bin/")
ls.split_dataset(
    start_date=start_train_date,
    last_train_date=split_Date,
    last_test_date=eval_date
)

Test date 2025-06-08 not found in the database. Omitting.


In [4]:
train_Xtree = ls.train_Xtree
train_ytree = ls.train_ytree
train_Xtime = ls.train_Xtime
train_ytime = ls.train_ytime

test_Xtree = ls.test_Xtree
test_ytree = ls.test_ytree
test_Xtime = ls.test_Xtime
test_ytime = ls.test_ytime

treenames = ls.featureTreeNames
timenames = ls.featureTimeNames

meta_pl_train = ls.meta_pl_train
meta_pl_test = ls.meta_pl_test

In [5]:
fs = FilterSamples(
    Xtree_train=train_Xtree,
    ytree_train=train_ytree,
    treenames=treenames,
    Xtree_test=test_Xtree,
    ytree_test=test_ytree,
    meta_train=meta_pl_train,
    meta_test=meta_pl_test,
    params=params
)

In [6]:
cat_mask_train, cat_mask_test = fs.categorical_masks()

In [ ]:
lincomb_mask_train, lincomb_mask_test = fs.lincomb_masks()
print(lincomb_mask_train.sum()/len(lincomb_mask_train))
print(lincomb_mask_test.sum()/len(lincomb_mask_test))

In [ ]:
taylor_mask_train, taylor_mask_test = fs.taylor_feature_masks()
print(taylor_mask_train.sum()/len(taylor_mask_train))
print(taylor_mask_test.sum()/len(taylor_mask_test))